1. Método para tratar o dado que tenha valor tanto para teste quanto para treino.
2. Método para submission preparation.
3. método para gerar arquivo de submissão.

# 0.0 IMPORT

In [573]:
import pandas as pd
import numpy as np
import json
import datetime
import ast
import dtale
import random

from sklearn import preprocessing as pp
import pickle

from sklearn import model_selection as ms
import category_encoders as ce

from boruta   import BorutaPy
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_validate
from sklearn.ensemble        import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from xgboost                 import XGBRegressor
from sklearn.linear_model    import LogisticRegression, SGDRegressor
from sklearn.neighbors       import KNeighborsRegressor
from sklearn.tree            import DecisionTreeRegressor
from sklearn.ensemble        import RandomForestRegressor, BaggingRegressor
from sklearn.metrics         import accuracy_score, recall_score, precision_score, balanced_accuracy_score, f1_score, roc_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes     import GaussianNB
from lightgbm                import LGBMRegressor

## 0.1 Function Train and Test

In [574]:
def load_data(url):
    # Opening JSON file
    train = open(url)

    # returns JSON object as a dictionary
    data_train_aux = json.load(train)

    data_train = pd.json_normalize(data_train_aux, record_path = 'data')
    data_train.columns = data_train_aux['columns']
    return data_train

In [575]:
def limpeza_inicial(df):
    
    if 'actual_price' in df.columns:
        df = df.drop(columns=['description', 'images','crawled_at','title'])
        prices = df[['seller','pid', 'actual_price']].groupby('pid').max().reset_index()
    
        precos_nulos = df.loc[df['actual_price'].isna(), ['_id','pid']]

        precos_recuperados = pd.merge(precos_nulos, prices, how = 'inner', on = 'pid')

        df_aux = pd.merge(df, precos_recuperados[['_id', 'actual_price']], on = '_id', how = 'left')
        df['actual_price'] = df_aux['actual_price_x'].fillna(0) + df_aux['actual_price_y'].fillna(0)

        df = df.dropna(subset=['actual_price'])
        
    else:
        df = df.drop(columns=['description', 'images','crawled_at','title'])

    return df

In [576]:
def f(x):
    try:
        return ast.literal_eval(str(x))   
    except Exception as e:
        print(e)
        return []

In [577]:
def junta_dict(dict_list):
        dicionario = {}
        for d in dict_list:
            for key, value in d.items():
                dicionario[key] = value
        return dicionario

In [578]:
def expand_product_detais (df):
    df_pd = df['product_details'].apply(lambda x: f(x))
    
    df_pd = pd.DataFrame([junta_dict(row) for row in df_pd], index = df_pd.index)
    
    df_pd = df_pd.drop(columns=['', ' '])

    # Removendo valores faltantes acima de 50%
    limite_nulos = len(df_pd) * 0.5  
    df_pd = df_pd.dropna(thresh = limite_nulos, axis=1)
    
    return df_pd


In [579]:
def df_pd_plus_df(df, df_pd):
    df = pd.concat([df, df_pd], axis=1)
    df = df.drop( ['product_details'] , axis = 1)
    return df

In [580]:
def ajuste_colunas (df):
    
    df.loc[df['Fabric'].isnull(), 'Fabric'] = 'outros'
    df.loc[df['Pattern'].isnull(), 'Pattern'] = 'outros'
    df.loc[df['Style Code'].isnull(), 'Style Code'] = 'outros'
    df.loc[df['Pack of'].isnull(), 'Pack of'] = 'outros'
    df.loc[df['Type'].isnull(), 'Type'] = 'outros'
    df.loc[df['Sleeve'].isnull(), 'Sleeve'] = 'outros'
    df.loc[df['Fit'].isnull(), 'Fit'] = 'outros'
    df.loc[df['Ideal For'].isnull(), 'Ideal For'] = 'outros'
    df.loc[df['Suitable For'].isnull(), 'Suitable For'] = 'outros'
    df.loc[df['Reversible'].isnull(), 'Reversible'] = 'outros'
    df.loc[df['Fabric Care'].isnull(), 'Fabric Care'] = 'outros'
    
    # # Suponha que você tenha uma coluna chamada 'Fabric' com valores do tipo object
    # df_new['Fabric'] = df_new['Fabric'].astype(str)
    # df_new['Pattern'] = df_new['Pattern'].astype(str)
    # df_new['Style Code'] = df_new['Style Code'].astype(str)
    # df_new['Pack of'] = df_new['Pack of'].astype(str)
    # df_new['Type'] = df_new['Type'].astype(str)
    # df_new['Sleeve'] = df_new['Sleeve'].astype(str)
    # df_new['Fit'] = df_new['Fit'].astype(str)
    # df_new['Ideal For'] = df_new['Ideal For'].astype(str)
    # df_new['Suitable For'] = df_new['Suitable For'].astype(str)
    # df_new['Reversible'] = df_new['Reversible'].astype(str)
    # df_new['Fabric Care'] = df_new['Fabric Care'].astype(str)


    # # Agora você pode usar o método str.split para dividir os valores
    # df_new['Fabric'] = df_new['Fabric'].str.split(',')
    # df_new['Pattern'] = df_new['Pattern'].str.split(',')
    # df_new['Style Code'] = df_new['Style Code'].str.split(',')
    # df_new['Pack of'] = df_new['Pack of'].str.split(',')
    # df_new['Type'] = df_new['Type'].str.split(',')
    # df_new['Sleeve'] = df_new['Sleeve'].str.split(',')
    # df_new['Fit'] = df_new['Fit'].str.split(',')
    # df_new['Ideal For'] = df_new['Ideal For'].str.split(',')
    # df_new['Suitable For'] = df_new['Suitable For'].str.split(',')
    # df_new['Reversible'] = df_new['Reversible'].str.split(',')
    # df_new['Fabric Care'] = df_new['Fabric Care'].str.split(',')
    
    return df

In [581]:
def type_sex(df):

# Dados sem virgulas: Style Code, Pack of, Suitable For, Reversible, 'Ideal For'


# Ideal for: 'Men', 'Men, Boys'/ 'Boys, Men' = 'all_ages',
            # 'Boys, Girls, Men, Women' = 'unisex_all_ages', 'Women, Men' = 'Unisex'
    
    df['Ideal For'] = df['Ideal For'].apply(lambda x : 'men' if x == 'Men'
                                            else 'all_ages' if x == 'Men, Boys'
                                            else 'all_ages' if x == 'Boys, Men'
                                            else 'unisex_all_ages' if x == 'unisex_all_ages'
                                            else 'unisex' if x == 'Women, Men'
                                            else 'outros')
    
    return df

In [582]:
def feature_engine (df):
    
    ## out_of_stock - transformar para int 
    df['out_of_stock'] = df['out_of_stock'].astype('int64')

    # brand - substituir por outros
    df['brand'] = df['brand'].apply(lambda x: x.lower())

    # criando feature product
    df['product'] = df[['category', 'sub_category']].apply(lambda x: x['category'] + '_' + x['sub_category'], axis = 1)
    
    return df


In [583]:
def split_df(df):
    
    if 'actual_price' in df.columns:
        X = df.drop(columns=['_id','pid','actual_price'], axis=1)
        y = df['actual_price']
        X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.1)
        data_train = pd.concat([X_train, y_train], axis=1)
        data_test = pd.concat([X_test, y_test], axis=1)
        
        return X_train, X_test, y_train, y_test
    else:
        X_prod = df.drop(columns=['_id','pid'], axis=1)
        return X_prod 


In [584]:
def var_resposta(y_train):
    # Tranformando a variável resposta em logaritmica
    y_train = np.log1p(y_train)
    return y_train

In [585]:
def rescaling_train(df_train):
    
    # 'avg_delivery_time_days',
    mm = pp.MinMaxScaler()

    df_train['avg_delivery_time_days'] = mm.fit_transform(df_train[['avg_delivery_time_days']])
    pickle.dump(mm, open('parameters/mm_avg_delivery_time_days.pkl','wb'))

    rs_average_rating = pp.RobustScaler()
    rs_number_of_reviews = pp.RobustScaler()

    df_train['average_rating'] = rs_average_rating.fit_transform(df_train[['average_rating']])
    pickle.dump(rs_average_rating, open('parameters/rs_average_rating.pkl','wb'))

    df_train['number_of_reviews'] = rs_number_of_reviews.fit_transform(df_train[['number_of_reviews']])
    pickle.dump(rs_number_of_reviews, open('parameters/rs_number_of_reviews.pkl','wb'))
    
    return df_train

    ##### Fazer Robust Scaler da variável resposta (??) #####

def rescaling_test(df_test):
    
    mm = pickle.load(open('parameters/mm_avg_delivery_time_days.pkl','rb'))
    df_test['avg_delivery_time_days'] = mm.transform(df_test[['avg_delivery_time_days']])
    
    rs_ar = pickle.load(open('parameters/rs_average_rating.pkl','rb'))
    df_test['average_rating'] = rs_ar.transform(df_test[['average_rating']])
    
    rs_nr = pickle.load(open('parameters/rs_number_of_reviews.pkl','rb'))
    df_test['number_of_reviews'] = rs_nr.transform(df_test[['number_of_reviews']])
    
    return df_test

In [586]:
def tranform_train(X_train, y_train):
    
    #FREQUENCY ENCODER
    for att in ['brand',
                'product',
                'seller',
                'category',
                'Suitable For',
                'Reversible',
                'Ideal For',
                'out_of_stock',
                'Style Code',
                'Pack of']:
        fe =  ce.CountEncoder(cols=[att], normalize=True).fit(X_train)
        pickle.dump(fe,open(f'parameters/fe_{att}.pkl', 'wb'))
        X_train = fe.transform(X_train)
    
    # Instancie o codificador alvo para 'Style Code'
    #te_style_code = ce.TargetEncoder()
    #te_pack_of = ce.TargetEncoder()
    
    #X_train['Style Code'] = te_style_code.fit_transform(X_train['Style Code'], y_train) 
    #pickle.dump(te_style_code, open('parameters/te_style_code.pkl', 'wb'))

    # Instancie o codificador alvo para 'Pack of'
    #X_train['Pack of'] = te_pack_of.fit_transform(X_train['Pack of'], y_train) 
    #pickle.dump(te_pack_of, open('parameters/te_pack_of.pkl', 'wb'))
    
    return X_train

def tranform_test(X_test):
    
    # FREQUENCY ENCODER
    for att in ['brand',
                'product',
                'seller',
                'category',
                'Suitable For',
                'Reversible',
                'Ideal For',
                'out_of_stock',
                'Style Code',
                'Pack of']:
        fe = pickle.load(open(f'parameters/fe_{att}.pkl', 'rb'))
        X_test = fe.transform(X_test)

    # TARGET ENCODE
    #te_style_code = pickle.load(open('parameters/te_style_code.pkl','rb'))
    #X_test['Style Code'] = te_style_code.transform(X_test['Style Code'])

    #te_pack_of = pickle.load(open('parameters/te_pack_of.pkl','rb'))
    #X_test['Pack of'] = te_pack_of.transform(X_test['Pack of'])

    return X_test

In [587]:
def feature_select_LGBM (X_train, y_train):
    
    # Crie e treine o modelo LGBMClassifier
    lgb_model = LGBMRegressor(n_jobs=-1, random_state=42)
    lgb_model.fit(X_train, y_train)
    # Obtenha as importâncias das características do modelo
    importances = lgb_model.feature_importances_

    # Ordene as características por importância decrescente
    indices = np.argsort(importances)[::-1]

    # Imprima o ranking das características
    print('Feature ranking')

    df = pd.DataFrame()
    for i, j in zip(X_train.columns, importances):
        aux = pd.DataFrame({'feature': i, 'importance': j}, index=[0])

        df = pd.concat([df, aux], axis=0)
        df = df.sort_values('importance', ascending=False)
    return df

In [588]:
def feature_selected_BORUTA(X_train, y_train):
    rf = RandomForestRegressor()
    
    # Define Boruta - Essa peste demora 1 hora ou mais.
    boruta = BorutaPy( rf, n_estimators = 'auto', verbose=2, random_state=42).fit( X_train, y_train)

    cols_selected = boruta.support_.tolist()
    print(cols_selected)

In [606]:
def feature_selected_manual (df):
    features_selected = ['average_rating',
    #'number_of_reviews',
    'brand',
    # 'category',
    # 'crawled_at',
    'out_of_stock',
    'avg_delivery_time_days',
    #  'product_details',
    'seller',
    #  'sub_category',
    #  'fabrication_time',
    #  'title',
    #  'Fabric',
    #  'Pattern',
    'Style Code',
    'Pack of',
    #  'Type',
    #  'Sleeve',
    #  'Fit',
    'Ideal For',
    'Suitable For',
    'Reversible',
    #  'Fabric Care',
    'product']
    
    return df[features_selected]

In [590]:
def MachineLearning(X_train, y_train, X_test, y_test):
    SEED = 42

    lista_de_medidas = ['SMAPE']
    nome_das_medidas = ['SMAPE']

    lista_de_modelos = [XGBRegressor(learning_rate=0.1, n_estimators=100, max_depth=3),
                        SGDRegressor(random_state=SEED), 
                        LGBMRegressor(random_state=SEED, n_jobs=-1, force_row_wise=True, ),
                        DecisionTreeRegressor(random_state=SEED),
                        KNeighborsRegressor(n_neighbors=5,  weights='distance',n_jobs=-1),
                        BaggingRegressor(),
                        RandomForestRegressor(random_state=SEED)]

    nome_dos_modelos = ['XGBoost', 
                        'SGD', 
                        'LGBM', 
                        'DecisionTree',
                        'KNN',
                        'Bagging',
                        'RandomForest']

    resultados0 = {}
    



    for i in range(len(lista_de_modelos)):
        print('Rodando modelo: ' + nome_dos_modelos[i])
        
        model = lista_de_modelos[i]
        
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        
        smape = 100 / len(y_test) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test)) / (np.abs(np.expm1(y_test)) + np.abs(np.expm1(y_pred))))
        # Calculo meigarom: 100 / len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred)))

        resultados0[nome_dos_modelos[i]] = [smape]
    
    resultados = pd.DataFrame(resultados0, index = nome_das_medidas).T

    return resultados

In [591]:
def cross_validation (model_name, X_training, kfold, model, verbose = False ):
    smape_list = []
    
    
    # Star an end date validation
    for k in reversed( range (1, kfold) ):
        if verbose:
            print('\nKfold: {}'.format(k))
        validation_model_start = (len(X_training)/(kfold-k))
        validation_model_end =  (len(X_training)) - (validation_model_start)

        # Filter Data Set
        treino = X_training.head(validation_model_start)
        validacao = X_training.tail(validation_model_end)

        # Training and validation dataset
        # Treino
        xtreino = treino.drop( ['actual_price'] , axis = 1)
        ytreino = treino['actual_price']

        # Validação
        xvalidacao = validacao.drop( ['actual_price'] , axis = 1)
        yvalidacao = validacao['actual_price']

        # Modelo
        m = model.fit( xtreino, ytreino )

        # Predição
        yhat_m = m.predict( xvalidacao )
        
        smape = 100 / len(yvalidacao) * np.sum(2 * np.abs(np.expm1(yhat_m) - np.expm1(yvalidacao)) / (np.abs(np.expm1(yvalidacao)) + np.abs(np.expm1(yhat_m))))
        # Calculo meigarom: 100 / len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred)))
        
        # Guardando as performaces
        smape_list.append( smape )
        
        
    resposta_modelo = pd.DataFrame( {'Model Name': model_name,
                                     'SMAPE': smape_list},
                                   index = [0])
    
    # Fazendo somente por questões de aprendizagem
    #curiosidade = pd.DataFrame({'Iteracao' : iteracao,
    #                            'Treino Início' : treino_inicio,
    #                            'Treino Fim' : treino_fim,
    #                            'Validação Início' : validação_inicio,
    #                            'Validação Fim' : validação_fim} )
    return resposta_modelo #, curiosidade

In [592]:
# def objective_cv(trial):  
    
#     n_estimators = trial.suggest_int('n_estimators',20, 100)
#     eta = trial.suggest_float('eta',0.01, 0.05)
#     max_depth = trial.suggest_int('max_depth', 3, 9)
#     subsample = trial.suggest_float('subsample',0.1, 0.9)
#     colsample_bytree = trial.suggest_float('colsample_bytree',0.1,0.9)

#     # model definition
#     xgb_model = XGBRegressor( n_estimators    = n_estimators, 
#                               eta              = eta, 
#                               max_depth        = max_depth, 
#                               subsample        = subsample,
#                               colsample_bytree = colsample_bytree
#                              )
    
#     xgb_model.fit(X_train[features_selected], y_train)
    
#     # Model Prediction 
#     y_pred = xgb_model.predict(X_test_mod[features_selected])

#     # calculando rmse
#     smape = 100 / len(y_test_mod) * np.sum(2 * np.abs(np.expm1(y_pred) - np.expm1(y_test_mod)) / (np.abs(np.expm1(y_test_mod)) + np.abs(np.expm1(y_pred))))

#     return smape

In [593]:
# import optuna
# study = optuna.create_study(direction='minimize')
# study.optimize(objective_cv, n_trials=50)

In [594]:
def fine_tunnung (X_train, y_train): 
    #RANDON SEARCH
    param = {'n_estimators' : [1500, 1700, 3000, 3500],
         'eta' : [0.01, 0.03],
         'max_depth' : [3, 5, 9],
         'subsample': [0.1, 0.5, 0.7],
         'colsample_bytree' : [0.3, 0.7, 0.9],
         'min_child_weight' : [3, 8, 15]}

    MAX_EVAL = 10
    
    final_result = pd.DataFrame()
    for i in range (MAX_EVAL):
        # Escolher os parâmetros aleatórios
        hp = { k: random.sample(v, 1)[0] for k, v in param.items() }
        print(hp)


        # model
        model_xgb = XGBRegressor(objective = 'reg:squarederror',
                                n_estimators=hp['n_estimators'],
                                eta = hp['eta'],
                                max_depth = hp['max_depth'],
                                subsample = hp['subsample'],
                                colsample_bytree = hp['colsample_bytree'],
                                min_child_weight = hp['min_child_weight'])
        # performance

        result = cross_validation (model_name = 'XGBRegressor',
                                   X_training = X_train,
                                   kfold = 5,
                                   model = model_xgb,
                                   verbose = True )
        final_result = pd.concat([final_result, result])
    final_result

In [595]:
def tratamento_inicial (df):
    
    df = limpeza_inicial (df)
    df_pd = expand_product_detais (df)
    df = df_pd_plus_df(df, df_pd)
    df = ajuste_colunas (df)
    df = type_sex(df)
    df = feature_engine (df)
    
    return df

In [596]:
def best_model(model, x_test):
    
    yhat = np.expm1(model.predict(x_test))
    
    return yhat

## 0.3 Function Subission CSV

In [597]:
def submission(df_sub,yhat):
    # Preparando os dados para a submissão

    df_submission = pd.DataFrame()
    df_submission['pid'] = df_sub['pid']
    df_submission['actual_price'] = yhat

    # Exportando como CSV
    df_submission.to_csv('submissions/submission_g01.csv', index=False)

## 0.4 Load Data

In [598]:
url_treino = '../2023-10-20-Hackday_6/dataset/raw/train.json'
url_prod = '../2023-10-20-Hackday_6/dataset/raw/test.json'

df_treino = load_data(url_treino)
df_prod = load_data(url_prod)

# 1.0 Run Train Test

In [599]:
df_treino = tratamento_inicial(df_treino)

In [600]:
df_prod = tratamento_inicial(df_prod)

In [601]:
X_train, X_test, y_train, y_test = split_df(df_treino)
y_train = var_resposta(y_train)
y_test = var_resposta(y_test)

In [602]:
X_prod = split_df(df_prod)

In [603]:
X_train = rescaling_train(X_train)
X_test = rescaling_test(X_test)
X_prod = rescaling_test(X_prod)

In [604]:
X_train = tranform_train(X_train, y_train)
X_test = tranform_test(X_test)
X_prod = tranform_test(X_prod)

In [605]:
X_train = feature_selected_manual (X_train)
X_test = feature_selected_manual (X_test)
X_prod = feature_selected_manual (X_prod)

In [570]:
fetures = feature_select_LGBM (X_train, y_train)
fetures

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002354 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 592
[LightGBM] [Info] Number of data points in the train set: 18647, number of used features: 9
[LightGBM] [Info] Start training from score 6.152875
Feature ranking


,feature,importance
0,Style Code,964
0,average_rating,646
0,seller,503
0,brand,457
0,product,252
0,Pack of,148
0,out_of_stock,18
0,avg_delivery_time_days,12
0,number_of_reviews,0


In [ ]:
#feature_selected_BORUTA(X_train, y_train)

In [607]:
X_train = feature_selected_manual (X_train)
X_test = feature_selected_manual (X_test)
X_prod = feature_selected_manual (X_prod)

KeyError: "['Ideal For', 'Suitable For', 'Reversible'] not in index"

In [608]:
resultado = MachineLearning(X_train, y_train, X_test, y_test)
resultado

Rodando modelo: XGBoost
Rodando modelo: SGD
Rodando modelo: LGBM
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Total Bins 403
[LightGBM] [Info] Number of data points in the train set: 20978, number of used features: 9
[LightGBM] [Info] Start training from score 6.154553
Rodando modelo: DecisionTree
Rodando modelo: KNN
Rodando modelo: Bagging
Rodando modelo: RandomForest


,SMAPE
XGBoost,104.651421
SGD,104.491846
LGBM,104.648291
DecisionTree,96.443726
KNN,93.833053
Bagging,99.172782
RandomForest,99.360827


In [557]:
#fine_tunnung (X_train, y_train)

# 2.0 Submission Preparation

In [ ]:
best_params = {'n_estimators': 100,
 'eta': 0.04847968659990019,
 'max_depth': 7,
 'subsample': 0.31355832178198495,
 'colsample_bytree': 0.5456941887941139}

In [558]:
best_model = XGBRegressor(n_jobs = -1 , 
                                 random_state=0,
                                 n_estimators = best_params['n_estimators'],
                                 eta = best_params['eta'],
                                 max_depth = best_params['max_depth'],
                                 subsample = best_params['subsample'],
                                 colsample_bytree = best_params['colsample_bytree'])

# model fit
best_model.fit(X_train[features_selected], y_train)

In [559]:
yhat = best_model(best_model, X_prod)

# 3.0 Submission CSV

In [560]:
submission(df_prod,yhat)